<img src="./images/fancy_hashcat.png" alt="A cat with a tophat at a computer. Aka JtR + H" width="800"/>

# Advanced Techniques for Cracking CMIYC_2023 Hashes

### Disclaimer

**As this Notebook demonstrates how to create custom hashlists/wordlists, if you run this entire notebook it will create those files in the ./challenge_files/CMIYC2023_Street directory**

### References (And Spoilers)

I wrote three blog posts talking about how to use JupyterLab to aid in compeating in the CMIYC2023 Password Cracking Competition. These posts use a very early version of this framework and I'll be using the techniques descibed in them in this Notebook.

- [Part 1: Basic tips on how to use JupyterLab. The previous Notebook covers most of this](https://reusablesec.blogspot.com/2023/08/using-jupyterlab-to-manage-password.html)
- [Part 2: Creating custom hashlists and wordlist](https://reusablesec.blogspot.com/2023/08/using-jupyterlab-to-manage-password_22.html)
- [Part 3: Advanced Hashcat techniques](https://reusablesec.blogspot.com/2023/08/hashcat-tips-and-tricks-for-hacking.html)

### Loading the configuration and the initial challenge files

Even if you already did this in the previous Notebook, these Notebooks do not share a common kernel so you will need to reload the challenge files to make use of them in this Notebook

In [1]:
from lib_framework.session_mgr import SessionMgr

# Loading the config this way to make it os independent on what you are running this on
import os
import sys
config_file = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2023_Street',
    'config.yml'
    )

sm = SessionMgr(config_file)

Starting to load challenge yaml file. This may take a minute or two
Done loading the challenge yaml file.


### Loading cracked passwords from Potfiles

Just like loading the hashes, we also need to load the cracked passwords. This is very important for the following steps since we will be creating dictionaries and left lists which requires a few cracked passwords to be helpful. As in the previous Notebook, I'm seperating out the "loading pot files" from the "loading challenge files" since I reload the potfiles constantly during cracking sessions as I crack more passwords.                                                   

In [2]:
sm.load_main_pots(verbose=False, update_only=True)
sm.update_main_pots()

Number of new plains added to the JtR pot file: 0
Number of new plains added to the Hashcat pot file: 0


### Creating custom left lists (of hashes) based on metadata

With unsalted hashes, it generally doesn't matter if you have a bunch of "junk" hashes that you aren't directly targeting in the hashlists that you are cracking. But for salted and computationally expensive hashes, you want to only target hashes that your attacks have been optimized to crack. That's where the idea of targeted "left lists" come from.

For example, in the CMIYC 2023 competition, hashed belonging to employees in the Sales department were created with fairly easy to guess passwords. Therefore, let's create a left list that only contains hashes belonging to Sales teammembers. To do this, we'll use the **SessionMgr.create_left_list(is_jtr=True, file_name=None, hash_type=None, filter=None)** function.

**is_jtr**: If True, it will format the hashes for John the Ripper cracking sessions. If False it will format them for HashCat.

**file_name**: The name/location of the file to write the left-list. If not specified it will output them to stdout.

**hash_type**: Allows you to filter the left-list by hashtype. Aka if you want to only create it for bcrypt hashes. If None, then it will output uncracked hashes for all hash types

**filter**: A dictionary that contains all the key/value pairs to filter the left list on. If a value is listed as none, it will only filter based on if the key exists for a target containing the uncracked hash. You can specify multiple filters, but currently only one filter per key/value pair. Aka if you want key1:value1 and key1:value2, you can't do that and will need to run this twice and manually combine the results.

Note: If you forget what the categories are that you can filter on, you can always call SessionMgr.print_metadata_categories()

Note 2: You'll notice the **semicolon ;** at the end of the sm.create_left_list in the call below. That supresses printing the return value of the function to this Notebook. I'm doing this since create_left_list also returns a list of all the hashes written to disk as its return value as well. This enables more advanced usage of these commands which will be demostrated later in this Notebook.

In [3]:
# Writing the left list to the CMIYC2023_Street directory
file_name = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2023_Street',
    'sales_only.hash'
    )

sm.create_left_list(format="jtr", file_name=file_name, filter={'Department':'Sales'});

### Creating custom wordlists/dictionaries

The other thing you often need to do is create wordlists/dictionaries based on previously cracked passwords. This can be used as part of loopback style attacks, but you can also feed these wordlists into other analysis tools like the [PCFG Toolset](https://github.com/lakiw/pcfg_cracker) or [PACK](https://github.com/iphelix/pack).

One challenge with these competitions is that passwords from each challenge is often generated in a very different manner. This means that if you train on every cracked password it can create rules that don't apply to your targeted list. Therefore the framework's method to generate custom wordlists **SessionMgr.create_cracked_list(self, file_name=None, hash_type=None, filter=None)** has the same filtering ability/format as the method to **create_left_list()**. The only difference is that while create_left_list() generates a list of uncrackes hashes, create_cracked_list() creates a wordlist of cracked plaintext values.

In [4]:
# Writing the word list to the CMIYC2023_Street directory
file_name = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2023_Street',
    'sales_only.dic'
    )

sm.create_cracked_list(file_name=file_name, filter={'Department':'Sales'});

### Advanced Hashcat Association Attack Generation

Now let's get into more advanced use-cases. Specifically let's dig into generating custom wordlists and hashlists to support a Hashcat association attack **(-a9 mode)**. This is detailed more in this blog post (also listed above) [Advanced Hashcat Techniques](https://reusablesec.blogspot.com/2023/08/hashcat-tips-and-tricks-for-hacking.html). For CMIYC2023 a set of passwords were generated by converting the "Created" timestamp to a Unix epoc timestamp. To speed things up when cracking a high value/high cost hash like BCrypt, Hashcat can run a 1 to 1 attack of hash/guess which is what an association attack does. To generate a matching hashlist + wordlist for this you can do the following:

#### Step 1: Create the hash_list

The following code will create a left list of all uncracked bcrypt hashes. You'll notice we are also saving the results of this call to the variable bcrypt_hashes. This way we can later generate a matching wordlist. Also you'll notice we're filtering it so only hashes with a "Created" metadata field are being saved. Since it is **{"Created":None)** with **None** as the value it will save all hashes that have the field regardless of the field's value.

In [5]:
# Writing the left list to the CMIYC2023_Street directory
file_name = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2023_Street',
    'epoc_bcrypt.hash'
    )

sm.create_left_list(format="jtr", file_name=file_name, hash_type="bcrypt", filter={"Created":None})

['4:$2a$08$aS2yUSnLXC20LhjBXirILu.DfORtGv9sMXl3cQ8sHXFAIigD70jBq',
 '10:$2a$08$cEflaFnAPhP/WybRbUzKQOyNyDOPYfjHGHmYNAng85V5n4EuFCjsy',
 '19:$2a$08$R1O2UBXJKEjFKjHHKzjMPeszZLnAe4EloGBcp9XUJY41aST0tbqo.',
 '31:$2a$08$UlnKUVHDXjLJTlb2aVfvQuEz1hKU3J12U5DqYICDt81j8scCIiXkO',
 '34:$2a$08$b1H0K0KyJxKwSEPlQzjQTOuCtos03xteJ1yzQHpTC8dzYG3o4E0L6',
 '46:$2a$08$ZCjqXlP3Uk3pPw3VTS2wOuDhP3x6Za3saEbNWtNQufvqNMSUOieLW',
 '55:$2a$08$LhPrbkXYLBb1cUXlL0XzKO8aYveFGsGCJrWUTAK3kEiqFnJEW0YhC',
 '57:$2a$08$MCnRKjfkRkW1LTbEUS22Le.RRnJHXuN15JOZaJdEHNSkG.qww3zj6',
 '86:$2a$08$YVXBYETGYRTyQDXmThbPXuk2.JX1.A4Qo4o.sSkHLGy4TZ20Ng6cW',
 '88:$2a$08$ZSPlaCWsOi7WUjH0PEjOaee8DSUuJbn.XYjNF/df9H0UbeQ/pf9rW',
 '108:$2a$08$Rx/vKx.2YzHLWVn2WybpQua2/S9vuhbfDBGEV3iPNWwF2l/z8znLu',
 '112:$2a$08$OU7GbU6vYifUW1DOTxbyZO6vAtcI1GV/IUnklBvox4PsxDGavgSqq',
 '136:$2a$08$Oy2sckHKWVDpbCnKL1a3T.PgyXHaU0w2g20XwSnkTjDt4Od.QD/Zu']

#### Step 2: Create the wordlist


This example is going to look more like traditional Python3 code since there is no way that we can anticipate the weird plaintext generation methods we'll run into like using the Unix epoc time of the Created date. Still since JupyterLab lets us write code directly into these cells it makes it really easy to throw something together and troubleshoot it on the fly. Note, you need to run Step 1 first for the bcrypt_hashes variable to be initalized.

In [6]:
# Using Python's built-in datetime to do the time to Unix epoc conversion
import datetime

# Rather than have a left list formatted for a password cracking program, you can also create one
# of the hash indexes in this framework. That makes it easier to use them in other python code like
# the following. You do this by setting the format="index"
bcrypt_hashes = sm.create_left_list(format="index", file_name=None, hash_type="bcrypt", filter={"Created":None}, silent=True)

# Writing the wordlist list to the CMIYC2023_Street directory
file_name = os.path.join(
    '.',
    'challenge_files',
    'CMIYC2023_Street',
    'epoc_bcrypt.dic'
    )

# Open the file to write the wordlist to
with open (file_name, "w") as guess_output:
    # Go through each hash in the hashlist
    for hash_id in bcrypt_hashes:
        
        # This is a list of all targets that have this hash associated with them
        targets = sm.target_list.hash_lookup[hash_id]
        
        # Normally you would want to loop through all of the targets since theoretically the same
        # hash might be shared across multiple targets. But since hashcat's association attack requires a
        # 1 to 1 mapping we want to only have one guess per hash.
        if not targets:
            # Write a placeholder if there isn't a target for this hash. This shouldn't happen since
            # we filtered for hashes with the metadata "Created" which means they must have a target
            # associated with them. But it's always good to check twice!
            guess_output.write("placeholder")
            continue
        
        # As mentioned above, only look at the first target for this hash
        target_id = targets[0]
        if 'Created' in sm.target_list.targets[target_id].metadata:
            created_time = sm.target_list.targets[target_id].metadata['Created']
            
            # Get the timezone in the proper format to parse it with datetime
            # This was super annoying to figure out during a cracking session FYI
            gmt_timezone = created_time.replace("CST","-0600")
            gmt_timezone = gmt_timezone.replace("CDT","-0500")
            
            # Reorder the timestamp in the right format. Once again, very annoying to figure this out
            timestamp = datetime.datetime.strptime(gmt_timezone, '%a %b %d %H:%M:%S %z %Y')
            
            # Needed to adjust for daylight savings time. Super annoying. I'm sure there is
            # an easier way to do this
            epoc_timestamp = f"{int(datetime.datetime.strftime(timestamp, '%s')) + 3600}"
            
            # Actually save the results to disk
            guess_output.write(f"{epoc_timestamp}\n")
            
        else:
            # Write a placeholder if there isn't a created time for this hash. It shouldn't occur
            # since we created this hashlist with a "Created" filter
            guess_output.write("placeholder2")

#### Running the attack

Now that we have the left list **"epoc_bcrypt.hash"** and the wordlist **"epoc_bcrypt.dic"** we can now run an association attack with Hashcat. I'd recommend doing running that outside of this notebook but the command you can use on the command line is:

`hashcat -o cmiyc2023_hc.potfile -a 9 -m 3200 epoc_bcrypt.hash epoc_bcrypt.dic`

As a friendly reminder, Hashcat association attacks by defualt will not use your normal potfiles since they can generate duplicate guesses. Which is why a different potfile is specified in the command above.